**Install important libraries**

In [1]:
!pip install spotipy

**Import necessary libraries**

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests 
import base64
import spotipy # providing access to Spotif's web API
from spotipy.oauth2 import SpotifyOAuth # Authintication 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

 To get started with building a Music Recommendation System, we first need to have an access token.<Br> 
 The access token serves as a temporary authorization credential, allowing the code to make authenticated requests to the Spotify API on behalf of the application.<Br> 
 Below is how we can get it:

In [3]:
# Client id / secret for API
CLIENT_ID = 'a1a5be34d3f54269a8bd8064c2cfd749'
CLIENT_SECRET = '6b3ce265ffb84d06aec7373abdeca95f'

# Base64 encode the client id and secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token 
token_url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization' : f'Basic {client_credentials_base64.decode()}'}
data = {'grant_type':'client_credentials'}

response = requests.post(token_url , data=data , headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access Token Obtained SUCCESSFULLY!")
else:
    print("Error obtaining access token")
    exit()    

Access Token Obtained SUCCESSFULLY!


Now, I’ll write a function to get music data from any playlist on Spotify.<Br>
For this task, you need to install the Spotipy library, which is a Python library providing access to Spotify’s web API.<Br>
Here’s how to install it on your system by writing the command mentioned below in your command prompt or terminal:-<Br>

!pip install spotipy

In [5]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [6]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                    Track Name                                   Artists  \
0                  Not Like Us                            Kendrick Lamar   
1                      Houdini                                    Eminem   
2   BAND4BAND (feat. Lil Baby)                     Central Cee, Lil Baby   
3           I Don't Wanna Wait                 David Guetta, OneRepublic   
4                        Pedro   Jaxomy, Agatino Romero, Raffaella Carrà   
..                         ...                                       ...   
95           Edge of Seventeen                                      Wuki   
96        Marianela (Que Pasa)  HUGEL, Merk & Kremont, Lirico En La Casa   
97                  Rhyme Dust                             MK, Dom Dolla   
98                        Vent                                      NOIT   
99                        RIZZ                                     AYYBO   

                    Album Name                Album ID  \
0                  Not Like U

In [7]:
data = music_df

The weighted popularity score is computed based on the time span. <Br>
The formula to calculate the weight is 1 / (time_span.days + 1). <Br>
The time_span.days attribute of the timedelta object gives the number of days in the time span between the release date and today.<Br>
Adding 1 to the number of days ensures that the weight is never zero, even for very recent releases, as this would lead to a division by zero error.<Br>

---

The idea behind this formula is that the weight decreases as the time span between the release date and today increases. <Br> 
More recent releases will have a higher weight, while older releases will have a lower weight. <Br>
As a result, when combining this weighted popularity score with other factors in a recommendation system, recent tracks will have a more significant impact on the final recommendations, reflecting users’ potential interest in newer music.

In [12]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

### Now let’s normalize the music features before moving forward:

In [13]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [14]:
data

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Not Like Us,Kendrick Lamar,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,6AI3ezQ4o3HUoP6Dhudph3,96,2024-05-04,274192,True,https://open.spotify.com/track/6AI3ezQ4o3HUoP6...,...,0.472,1,-7.001,1,0.0776,0.0107,0.000000,0.1410,0.214,101.061
1,Houdini,Eminem,Houdini,6Xuu2z00jxRPZei4IJ9neK,2HYFX63wP3otVIvopRS99Z,94,2024-05-31,227239,True,https://open.spotify.com/track/2HYFX63wP3otVIv...,...,0.887,9,-2.760,0,0.0683,0.0292,0.000002,0.0582,0.889,127.003
2,BAND4BAND (feat. Lil Baby),"Central Cee, Lil Baby",BAND4BAND (feat. Lil Baby),4AzPr5SUpNF553eC1d3aRy,7iabz12vAuVQYyekFIWJxD,91,2024-05-23,140733,True,https://open.spotify.com/track/7iabz12vAuVQYye...,...,0.764,11,-5.241,1,0.2040,0.3590,0.000000,0.1190,0.886,140.113
3,I Don't Wanna Wait,"David Guetta, OneRepublic",I Don't Wanna Wait,0wCLHkBRKcndhMQQpeo8Ji,331l3xABO0HMr1Kkyh2LZq,90,2024-04-05,149668,False,https://open.spotify.com/track/331l3xABO0HMr1K...,...,0.714,1,-4.617,0,0.0309,0.0375,0.000000,0.2320,0.554,129.976
4,Pedro,"Jaxomy, Agatino Romero, Raffaella Carrà",Pedro,5y6RXjI5VPR0RyInghTbf1,48lxT5qJF0yYyf2z4wB4xW,89,2024-03-29,144846,False,https://open.spotify.com/track/48lxT5qJF0yYyf2...,...,0.936,9,-6.294,1,0.3010,0.0229,0.000001,0.3110,0.844,151.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Edge of Seventeen,Wuki,Edge of Seventeen,6oeEtPWpOfed7plMHI5qef,5IFSvSEeCAvvBEQ85poqKB,70,2022-10-21,164043,False,https://open.spotify.com/track/5IFSvSEeCAvvBEQ...,...,0.909,5,-5.999,0,0.0476,0.0324,0.527000,0.0800,0.351,127.997
96,Marianela (Que Pasa),"HUGEL, Merk & Kremont, Lirico En La Casa",Marianela (Que Pasa),5As1VmPUMn4HIgYSbFD6l0,5bZjb7xKqLqa58QiUBcVvl,69,2022-11-25,145766,False,https://open.spotify.com/track/5bZjb7xKqLqa58Q...,...,0.885,1,-4.184,1,0.0498,0.0353,0.006550,0.0785,0.532,124.041
97,Rhyme Dust,"MK, Dom Dolla",Rhyme Dust,1vSO4N7cwo4pBywCSGF5BR,59QDyqLww2pxyg9ijOPO7f,69,2023-02-24,181365,False,https://open.spotify.com/track/59QDyqLww2pxyg9...,...,0.561,3,-8.675,1,0.1760,0.0659,0.027100,0.1520,0.721,128.034
98,Vent,NOIT,Vent,5wr0mILtj1RIw3ig141YpU,4yMXKeuRDOIzrh3fBiJYds,69,2024-03-01,176898,True,https://open.spotify.com/track/4yMXKeuRDOIzrh3...,...,0.583,3,-9.584,0,0.3760,0.0295,0.001170,0.1620,0.954,126.183


### Hybird Recommendation System 
 - Content-Based filter
 - Collaborative filtering

- Collaborative filtering :-
 analyzes user-item interactions and identifies similarities between users or items to make recommendations.<Br>
 It recommends items users with similar preferences have liked or consumed. However, it may struggle with new or niche items having limited user interactions.<Br>

On the other hand,

- Content-based filtering :-
 focuses on features and characteristics of items to recommend similar items to users based on their preferences. <Br>
 It examines attributes like product descriptions, brands, categories, and user profiles.

In [15]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

****Now here’s the function to generate music recommendations based on weighted popularity and combine it with the recommendations of the content-based filtering method using the hybrid approach:****

In [20]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [22]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Hybrid recommended songs for 'I'm Good (Blue)':


C:\Users\win10\AppData\Local\Temp\ipykernel_13328\209674984.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({


,Track Name,Artists,Album Name,Release Date,Popularity
4,It's Not Right But It's Okay,Mr. Belt & Wezol,It's Not Right But It's Okay,2024-02-23,83.0
3,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,77.0
2,Where You Are,"John Summit, HAYLA",Where You Are,2023-03-03,75.0
0,In My Bones,"Lost Frequencies, David Kushner",In My Bones,2024-04-19,74.0
1,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,74.0
